# bref team batting

In [8]:
BREF_TEAM_BATTING_URL = "https://www.baseball-reference.com/teams/{team}/{year}-batting.shtml#players_standard_batting"

In [9]:
import requests
from bs4 import BeautifulSoup
import polars as pl

In [10]:
data = requests.get(test_url).text
soup = BeautifulSoup(data, "html.parser")
tables = soup.find_all("table")


In [11]:
# Assuming we are interested in the first table
table = tables[0]

# Extract headers
headers = [th.text for th in table.find("thead").find_all("th")]
headers.remove("Rk")
headers[-2] = "AllPos"
print(headers)
# Extract rows
rows = []
for row in table.find("tbody").find_all("tr"):
    if "class" not in row.attrs or row.attrs["class"] != ["thead"]:
        cells = row.find_all("td")
        cells = [cell.text.strip() for cell in cells]
        rows.append(cells)
print(rows)
# Create the dataframe using Polars
df = pl.DataFrame(rows, schema=headers, orient="row")
df = df.rename({"WAR": "bWAR"})
df = df.filter(pl.col("OPS+") != "")
df = df.with_columns(
    [
        pl.col("Age").cast(pl.Int32),
        pl.col("G").cast(pl.Int32),
        pl.col("bWAR").cast(pl.Float32),
        pl.col("PA").cast(pl.Int32),
        pl.col("AB").cast(pl.Int32),
        pl.col("R").cast(pl.Int32),
        pl.col("H").cast(pl.Int32),
        pl.col("2B").cast(pl.Int32),
        pl.col("3B").cast(pl.Int32),
        pl.col("HR").cast(pl.Int32),
        pl.col("RBI").cast(pl.Int32),
        pl.col("SB").cast(pl.Int32),
        pl.col("CS").cast(pl.Int32),
        pl.col("BB").cast(pl.Int32),
        pl.col("SO").cast(pl.Int32),
        pl.col("BA").cast(pl.Float32),
        pl.col("OBP").cast(pl.Float32),
        pl.col("SLG").cast(pl.Float32),
        pl.col("OPS").cast(pl.Float32),
        pl.col("OPS+").cast(pl.Int32),
        pl.col("rOBA").cast(pl.Float32),
        pl.col("Rbat+").cast(pl.Int32),
        pl.col("TB").cast(pl.Float32),
        pl.col("GIDP").cast(pl.Int32),
        pl.col("HBP").cast(pl.Int32),
        pl.col("SH").cast(pl.Int32),
        pl.col("SF").cast(pl.Int32),
        pl.col("IBB").cast(pl.Int32),
    ]
)
df


['Player', 'Age', 'Pos', 'WAR', 'G', 'PA', 'AB', 'R', 'H', '2B', '3B', 'HR', 'RBI', 'SB', 'CS', 'BB', 'SO', 'BA', 'OBP', 'SLG', 'OPS', 'OPS+', 'rOBA', 'Rbat+', 'TB', 'GIDP', 'HBP', 'SH', 'SF', 'IBB', 'AllPos', 'Awards']
[['Gabriel Moreno', '24', 'C', '2.6', '97', '351', '305', '39', '81', '18', '1', '5', '45', '3', '1', '41', '52', '.266', '.353', '.380', '.734', '105', '.327', '105', '116', '10', '2', '0', '3', '1', '2H/5', ''], ['Christian Walker', '33', '1B', '2.6', '130', '552', '479', '72', '120', '26', '0', '26', '84', '2', '2', '55', '133', '.251', '.335', '.468', '.803', '121', '.340', '116', '224', '12', '10', '0', '8', '4', '*3/H', 'GG'], ['Ketel Marte#', '30', '2B', '6.8', '136', '583', '504', '93', '147', '23', '2', '36', '95', '7', '1', '65', '106', '.292', '.372', '.560', '.932', '155', '.396', '155', '282', '9', '5', '0', '9', '7', '*4D/H6', 'AS,MVP-3,SS'], ['Geraldo Perdomo#', '24', 'SS', '3.5', '98', '388', '337', '61', '92', '21', '2', '3', '37', '9', '1', '36', '58',

Player,Age,Pos,bWAR,G,PA,AB,R,H,2B,3B,HR,RBI,SB,CS,BB,SO,BA,OBP,SLG,OPS,OPS+,rOBA,Rbat+,TB,GIDP,HBP,SH,SF,IBB,AllPos,Awards
str,i32,str,f32,i32,i32,i32,i32,i32,i32,i32,i32,i32,i32,i32,i32,i32,f32,f32,f32,f32,i32,f32,i32,f32,i32,i32,i32,i32,i32,str,str
"""Gabriel Moreno""",24,"""C""",2.6,97,351,305,39,81,18,1,5,45,3,1,41,52,0.266,0.353,0.38,0.734,105,0.327,105,116.0,10,2,0,3,1,"""2H/5""",""""""
"""Christian Walker""",33,"""1B""",2.6,130,552,479,72,120,26,0,26,84,2,2,55,133,0.251,0.335,0.468,0.803,121,0.34,116,224.0,12,10,0,8,4,"""*3/H""","""GG"""
"""Ketel Marte#""",30,"""2B""",6.8,136,583,504,93,147,23,2,36,95,7,1,65,106,0.292,0.372,0.56,0.932,155,0.396,155,282.0,9,5,0,9,7,"""*4D/H6""","""AS,MVP-3,SS"""
"""Geraldo Perdomo#""",24,"""SS""",3.5,98,388,337,61,92,21,2,3,37,9,1,36,58,0.273,0.344,0.374,0.718,101,0.328,104,126.0,5,3,7,5,0,"""6/H""",""""""
"""Eugenio SuÃ¡rez""",32,"""3B""",3.1,158,640,571,90,146,28,2,30,101,2,1,49,176,0.256,0.319,0.469,0.788,116,0.34,115,268.0,10,9,0,11,0,"""*5/D""",""""""
…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…
"""Adrian Del Castillo*""",24,"""C""",0.5,25,87,80,12,25,5,0,4,19,1,1,7,28,0.313,0.368,0.525,0.893,146,0.379,146,42.0,1,0,0,0,0,"""2/H""",""""""
"""Luis Guillorme*""",29,"""2B""",0.3,18,49,37,6,6,2,0,0,4,3,0,11,8,0.162,0.347,0.216,0.563,63,0.292,70,8.0,2,0,0,1,0,"""4/H5D""",""""""
"""Jace Peterson*""",34,"""IF""",-0.6,14,26,22,2,1,0,0,0,1,0,0,3,6,0.045,0.154,0.045,0.199,-40,0.114,-48,1.0,1,0,0,1,0,"""/H64D5""",""""""


In [14]:
def fangraphs_team_batting_url(team: str, year: int) -> str:
    data = requests.get(BREF_TEAM_BATTING_URL.format(team=team, year=year)).text
    soup = BeautifulSoup(data, "html.parser")
    tables = soup.find_all("table")
    table = tables[0]

    # Extract headers
    headers = [th.text for th in table.find("thead").find_all("th")]
    headers.remove("Rk")
    headers[-2] = "AllPos"
    # print(headers)
    # Extract rows
    rows = []
    for row in table.find("tbody").find_all("tr"):
        if "class" not in row.attrs or row.attrs["class"] != ["thead"]:
            cells = row.find_all("td")
            cells = [cell.text.strip() for cell in cells]
            rows.append(cells)
    # print(rows)
    # Create the dataframe using Polars
    df = pl.DataFrame(rows, schema=headers, orient="row")
    df = df.rename({"WAR": "bWAR"})
    df = df.filter(pl.col("OPS+") != "")
    df = df.with_columns(
        [
            pl.col("Age").cast(pl.Int32),
            pl.col("G").cast(pl.Int32),
            pl.col("bWAR").cast(pl.Float32),
            pl.col("PA").cast(pl.Int32),
            pl.col("AB").cast(pl.Int32),
            pl.col("R").cast(pl.Int32),
            pl.col("H").cast(pl.Int32),
            pl.col("2B").cast(pl.Int32),
            pl.col("3B").cast(pl.Int32),
            pl.col("HR").cast(pl.Int32),
            pl.col("RBI").cast(pl.Int32),
            pl.col("SB").cast(pl.Int32),
            pl.col("CS").cast(pl.Int32),
            pl.col("BB").cast(pl.Int32),
            pl.col("SO").cast(pl.Int32),
            pl.col("BA").cast(pl.Float32),
            pl.col("OBP").cast(pl.Float32),
            pl.col("SLG").cast(pl.Float32),
            pl.col("OPS").cast(pl.Float32),
            pl.col("OPS+").cast(pl.Int32),
            pl.col("rOBA").cast(pl.Float32),
            pl.col("Rbat+").cast(pl.Int32),
            pl.col("TB").cast(pl.Float32),
            pl.col("GIDP").cast(pl.Int32),
            pl.col("HBP").cast(pl.Int32),
            pl.col("SH").cast(pl.Int32),
            pl.col("SF").cast(pl.Int32),
            pl.col("IBB").cast(pl.Int32),
        ]
    )
    return df

In [15]:
fangraphs_team_batting_url("NYY", 2021)

Player,Age,Pos,bWAR,G,PA,AB,R,H,2B,3B,HR,RBI,SB,CS,BB,SO,BA,OBP,SLG,OPS,OPS+,rOBA,Rbat+,TB,GIDP,HBP,SH,SF,IBB,AllPos,Awards
str,i32,str,f32,i32,i32,i32,i32,i32,i32,i32,i32,i32,i32,i32,i32,i32,f32,f32,f32,f32,i32,f32,i32,f32,i32,i32,i32,i32,i32,str,str
"""Gary SÃ¡nchez""",28,"""C""",0.7,117,440,383,54,78,13,1,23,54,0,0,52,121,0.204,0.307,0.423,0.73,99,0.315,96,162.0,14,5,0,0,3,"""*2H/D""",""""""
"""Anthony Rizzo*""",31,"""1B""",0.5,49,200,173,32,43,7,0,8,21,2,0,16,28,0.249,0.34,0.428,0.768,110,0.354,122,74.0,5,9,0,2,0,"""3/H""",""""""
"""DJ LeMahieu""",32,"""2B""",1.5,150,679,597,84,160,24,1,10,57,4,2,73,94,0.268,0.349,0.362,0.711,97,0.317,98,216.0,16,4,0,5,2,"""435/HD""",""""""
"""Gleyber Torres""",24,"""SS""",0.8,127,516,459,50,119,22,0,9,51,14,6,50,104,0.259,0.331,0.366,0.697,93,0.312,94,168.0,12,1,2,4,1,"""64/H""",""""""
"""Gio Urshela""",29,"""3B""",0.6,116,442,420,42,112,18,2,14,49,1,0,20,109,0.267,0.301,0.419,0.72,96,0.313,95,176.0,16,1,0,1,0,"""56/H""",""""""
…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…
"""Clarke Schmidt""",25,"""P""",0.0,1,2,1,0,0,0,0,0,0,0,0,1,1,0.0,0.5,0.0,0.5,58,0.346,117,0.0,0,0,0,0,0,"""/1""",""""""
"""Corey Kluber""",35,"""P""",0.0,1,2,2,0,0,0,0,0,0,0,0,0,1,0.0,0.0,0.0,0.0,-100,0.0,-125,0.0,0,0,0,0,0,"""1""",""""""
"""Jameson Taillon""",29,"""P""",0.0,2,2,1,0,0,0,0,0,0,0,0,1,0,0.0,0.5,0.0,0.5,58,0.346,117,0.0,0,0,0,0,0,"""1""",""""""


In [16]:
fangraphs_team_batting_url("CHC", 1980)

Player,Age,Pos,bWAR,G,PA,AB,R,H,2B,3B,HR,RBI,SB,CS,BB,SO,BA,OBP,SLG,OPS,OPS+,rOBA,Rbat+,TB,GIDP,HBP,SH,SF,IBB,AllPos,Awards
str,i32,str,f32,i32,i32,i32,i32,i32,i32,i32,i32,i32,i32,i32,i32,i32,f32,f32,f32,f32,i32,f32,i32,f32,i32,i32,i32,i32,i32,str,str
"""Tim Blackwell#""",27,"""C""",3.0,103,366,320,24,87,16,4,5,30,0,1,41,62,0.272,0.352,0.394,0.745,103,0.338,98,126.0,5,0,2,3,6,"""2/H""",""""""
"""Bill Buckner*""",30,"""1B""",1.5,145,615,578,69,187,41,3,10,68,1,2,30,18,0.324,0.353,0.457,0.81,119,0.363,116,264.0,13,0,0,6,11,"""379/H""","""MVP-14"""
"""Mike Tyson""",30,"""2B""",-0.7,123,363,341,34,81,19,3,3,23,1,2,15,61,0.238,0.273,0.337,0.61,65,0.285,58,115.0,5,2,4,1,3,"""*4/H""",""""""
"""IvÃ¡n de JesÃºs""",27,"""SS""",1.9,157,692,618,78,160,26,3,3,33,44,16,60,81,0.259,0.327,0.325,0.653,78,0.32,79,201.0,4,4,8,2,2,"""*6/H""",""""""
"""Lenny Randle#""",31,"""3B""",1.4,130,549,489,67,135,19,6,5,39,19,13,50,55,0.276,0.343,0.37,0.713,94,0.337,98,181.0,13,1,7,2,2,"""*54H/78""",""""""
…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…
"""Bill Caudill""",23,"""P""",0.0,72,10,9,0,2,1,0,0,1,0,0,0,2,0.222,0.222,0.333,0.556,49,0.247,22,3.0,1,0,1,0,0,"""1""",""""""
"""Bruce Sutter""",27,"""P""",-0.1,60,10,9,0,1,0,0,0,1,0,0,0,6,0.111,0.111,0.111,0.222,-39,0.101,-97,1.0,0,0,1,0,0,"""1""","""AS"""
"""Randy Martz*""",24,"""P""",-0.1,6,9,9,0,1,0,0,0,0,0,0,0,5,0.111,0.111,0.111,0.222,-39,0.101,-77,1.0,0,0,0,0,0,"""/1""",""""""
